In [1]:
import tensorflow as tf
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd
import numpy as np
from tensorflow.python.client import device_lib
from keras.models import load_model
import gc
import onnx
import onnxruntime as rt

from mylib.featurenames import *
from mylib.cleaner import *
from mylib.scaler import *
from mylib.ftploader import *
from mylib.modelpostfix import *
from mylib.tester import *
from mylib.server_info import *

In [2]:
print(device_lib.list_local_devices())

engine = create_engine(mysql_server_uri)
conn = engine.connect()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12366587035919065121
xla_global_id: -1
]


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & (br_full_data['dTradeTime'] >= '2023-02-24')
br = br_full_data[get_filter]

In [5]:
''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y_condition = (br['fMaxPowerAfterBuyWhile30'] < 0.025)
y = np.where(y_condition, 1, 0)

y_test =y
y_predict = []

modelTester = ModelTester(engine, conn)

In [6]:
# 테스트할 모델

dnn_model_names = []

rf_model_names = ['rf_maxM_d3_v1', 'rf_maxM_d3_v2', 'rf_maxM_d2_v2', 'rf_maxM_d45_v2', 'rf_max30_d4_v2']
#['rf_max30_d3', 'rf_max30_d4', 'rf_max30_d5', 'rf_max30_d3_v2', 'rf_maxM_d3']
#['rf_s94_2000_6_v1', 'rf_s13_2000_6_v1', 'rf_s13(15_2000_6_v1', 
#'rf_pmd15_2000_6_v1', 'rf_pow30_01_2000_6_v1', 'rf_prf_01_2000_6_v1']

In [7]:
ftp= FtpLoader(ftp_ip, ftp_port, ftp_id, ftp_pw)

In [8]:

for idx, mn in enumerate(dnn_model_names):
    ftp.download(mn + h5_, h5_path, '/h5/')
    x_datas = modelTester.setFullScale(X, mn+onnx_)

    h5_model = load_model(h5_path + mn + h5_)

    pred = h5_model.predict(x_datas)
    y_predict.append(pred)
    
    del h5_model
    gc.collect()

In [9]:
for idx, mn in enumerate(rf_model_names):
    ftp.download(mn+onnx_, onnx_path, '/onnx/')
    
    x_datas = X

    # onnx_model = onnx.load(onnx_path + mn + onnx_)
    sess = rt.InferenceSession(onnx_path + mn + onnx_)
    res = sess.run(['output'],{'input': x_datas})
    y_predict.append(res[0])
    
    del sess
    gc.collect()

In [10]:
#################################
############## test #############
from mylib.tester import *
# suc_crit = 0.8 # 1이라 판정할 값의 기준
# fl_crit = 0.5  # 0이라 판정할 값의 기준

suc_ratio = 0.65 # 모델들에서 1이라 종합할 비율
fail_ratio = 0.65 # 모델들에서 0이라 종합할 비율

testClassification(y_test, y_predict, suc_pass_ratio=suc_ratio, fail_pass_ratio=fail_ratio)


총량 :  48862
0 :  14631 , 비율 :  29.94351438745856 (%)
1 :  34231 , 비율 :  70.05648561254144 (%)

============ predict 0 =============
총 횟수 :  3556 ,  타겟기준 :  0.0
실제 0 :  2166
실제 1 :  1390
정답비율 :  60.91113610798649 (%)

============ predict 1 =============
총 횟수 :  45306 , 타겟기준 :  1.0
실제 1 :  32841
실제 0 :  12465
정답비율 :  72.48708780294001 (%)



In [11]:
removeTestData(onnx_list=rf_model_names)